# PyTorch PCD Metrics - Package Tests

Test imports and basic functionality of all metric packages.

## 1. Environment Info and Test Data

In [1]:
from platform import python_version
import torch

print('Python version:', python_version())
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA version:', torch.version.cuda)
    print('GPU:', torch.cuda.get_device_name(0))

print('\n' + '='*50 + '\n')

# Generate test point clouds (reused across all tests)
if torch.cuda.is_available():
    a = torch.rand((32, 2048, 3)).cuda()
    b = torch.rand((32, 2048, 3)).cuda()
    print(f'Generated test data: a.shape={a.shape}, b.shape={b.shape}')
else:
    a = torch.rand((32, 2048, 3))
    b = torch.rand((32, 2048, 3))
    print(f'Generated test data (CPU): a.shape={a.shape}, b.shape={b.shape}')

Python version: 3.9.25
PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA version: 12.8
GPU: NVIDIA GeForce RTX 5080 Laptop GPU


Generated test data: a.shape=torch.Size([32, 2048, 3]), b.shape=torch.Size([32, 2048, 3])


## 2. Test Imports

In [2]:
# Test StructuralLosses
try:
    from StructuralLosses import compute_all_metrics, jsd_between_point_cloud_sets
    print('✓ StructuralLosses imported successfully')
    structural_losses_available = True
except ImportError as e:
    print('✗ StructuralLosses import failed:', e)
    structural_losses_available = False
    
# Test emd_wrapper
try:
    from emd_wrapper import emdModule, test_emd
    print('✓ emd_wrapper imported successfully')
    emd_available = True
except ImportError as e:
    print('✗ emd_wrapper import failed:', e)
    emd_available = False

# Test PyTorchEMD (emd)
try:
    from emd import earth_mover_distance, EarthMoverDistanceFunction
    print('✓ PyTorchEMD (emd) imported successfully')
    pytorchemd_available = True
except ImportError as e:
    print('✗ PyTorchEMD (emd) import failed:', e)
    pytorchemd_available = False

# Test ChamferDistance
try:
    from dist_chamfer_3D import chamfer_3DDist
    print('✓ ChamferDistance imported successfully')
    chamfer_available = True
except ImportError as e:
    print('✗ ChamferDistance import failed:', e)
    chamfer_available = False
    
# Test pytorch_pcd_metrics wrapper
try:
    from pytorch_pcd_metrics.structural_losses import compute_all_metrics as compute_all_metrics_wrapper
    from pytorch_pcd_metrics.emd import emdModule as emdModule_wrapper
    from pytorch_pcd_metrics.pytorchemd import earth_mover_distance as earth_mover_distance_wrapper
    from pytorch_pcd_metrics.chamfer import chamfer_3DDist as chamfer_3DDist_wrapper
    print('✓ pytorch_pcd_metrics wrapper imported successfully')
    wrapper_available = True
except ImportError as e:
    print('✗ pytorch_pcd_metrics wrapper import failed:', e)
    wrapper_available = False

✓ StructuralLosses imported successfully
✓ emd_wrapper imported successfully
✓ PyTorchEMD (emd) imported successfully
Loaded compiled 3D CUDA chamfer distance
✓ ChamferDistance imported successfully
✓ pytorch_pcd_metrics wrapper imported successfully


## 3. Test Modules (Direct)

In [3]:
# Test StructuralLosses
if structural_losses_available and torch.cuda.is_available():
    print('Testing StructuralLosses...\n')
    
    # Test compute_all_metrics
    results = compute_all_metrics(a, b, batch_size=32)
    print('L-GAN and 1-NN Metrics:')
    for key, value in results.items():
        print(f'  {key}: {value.item():.4f}')
    
    # Test JSD
    a_np = a.detach().cpu().numpy()
    b_np = b.detach().cpu().numpy()
    jsd = jsd_between_point_cloud_sets(a_np, b_np)
    print(f'\nJSD: {jsd:.6f}')
    
    print('\n✓ StructuralLosses test passed')
else:
    print('⊘ StructuralLosses test skipped (not available or no CUDA)')

print('\n' + '='*50 + '\n')

# Test EMD
if emd_available and torch.cuda.is_available():
    print('Testing EMD...\n')
    
    # Test EMD
    emd = emdModule()
    dist, assignment = emd(a, b, eps=0.002, iters=10000)
    
    print(f'EMD Results:')
    print(f'  dist shape: {dist.shape}')
    print(f'  assignment shape: {assignment.shape}')
    print(f'  Mean distance: {dist.mean().item():.6f}')
    print(f'  Std distance: {dist.std().item():.6f}')
    
    print('\n✓ EMD test passed')
else:
    print('⊘ EMD test skipped (not available or no CUDA)')

print('\n' + '='*50 + '\n')

# Test PyTorchEMD
if pytorchemd_available and torch.cuda.is_available():
    print('Testing PyTorchEMD...\n')
    
    # Test PyTorchEMD
    a_transpose = a.transpose(1, 2)  # Convert to (B, 3, N) format
    b_transpose = b.transpose(1, 2)
    cost = earth_mover_distance(a_transpose, b_transpose, transpose=True)
    
    print(f'PyTorchEMD Results:')
    print(f'  cost shape: {cost.shape}')
    print(f'  Mean cost: {cost.mean().item():.6f}')
    print(f'  Std cost: {cost.std().item():.6f}')
    print(f'  Min cost: {cost.min().item():.6f}')
    print(f'  Max cost: {cost.max().item():.6f}')
    
    print('\n✓ PyTorchEMD test passed')
else:
    print('⊘ PyTorchEMD test skipped (not available or no CUDA)')

print('\n' + '='*50 + '\n')

# Test ChamferDistance
if chamfer_available and torch.cuda.is_available():
    print('Testing ChamferDistance...\n')
    
    # Test ChamferDistance
    chamfer_dist = chamfer_3DDist()
    dist1, dist2, idx1, idx2 = chamfer_dist(a, b)
    
    print(f'ChamferDistance Results:')
    print(f'  dist1 shape: {dist1.shape}')
    print(f'  dist2 shape: {dist2.shape}')
    print(f'  idx1 shape: {idx1.shape}')
    print(f'  idx2 shape: {idx2.shape}')
    print(f'  Mean dist1: {dist1.mean().item():.6f}')
    print(f'  Mean dist2: {dist2.mean().item():.6f}')
    
    print('\n✓ ChamferDistance test passed')
else:
    print('⊘ ChamferDistance test skipped (not available or no CUDA)')

Testing StructuralLosses...

L-GAN and 1-NN Metrics:
  lgan_mmd-CD: 0.0044
  lgan_cov-CD: 0.5938
  lgan_mmd_smp-CD: 0.0044
  lgan_mmd-EMD: 0.0776
  lgan_cov-EMD: 0.5938
  lgan_mmd_smp-EMD: 0.0777
  1-NN-CD-acc_t: 0.3125
  1-NN-CD-acc_f: 0.5312
  1-NN-CD-acc: 0.4219
  1-NN-EMD-acc_t: 0.3750
  1-NN-EMD-acc_f: 0.3125
  1-NN-EMD-acc: 0.3438

JSD: 0.008095

✓ StructuralLosses test passed


Testing EMD...

EMD Results:
  dist shape: torch.Size([32, 2048])
  assignment shape: torch.Size([32, 2048])
  Mean distance: 0.004985
  Std distance: 0.005401

✓ EMD test passed


Testing PyTorchEMD...

PyTorchEMD Results:
  cost shape: torch.Size([32])
  Mean cost: 43.815937
  Std cost: 8.385543
  Min cost: 30.063068
  Max cost: 68.109688

✓ PyTorchEMD test passed


Testing ChamferDistance...

ChamferDistance Results:
  dist1 shape: torch.Size([32, 2048])
  dist2 shape: torch.Size([32, 2048])
  idx1 shape: torch.Size([32, 2048])
  idx2 shape: torch.Size([32, 2048])
  Mean dist1: 0.002290
  Mean dist2: 0

## 4. Test Wrapper Package

In [4]:
if wrapper_available and torch.cuda.is_available():
    print('Testing pytorch_pcd_metrics wrapper...\n')
    
    # Test StructuralLosses through wrapper
    print('1. Testing StructuralLosses through wrapper:')
    results_wrapper = compute_all_metrics_wrapper(a, b, batch_size=32)
    print('\nL-GAN and 1-NN Metrics (via wrapper):')
    for key, value in results_wrapper.items():
        print(f'  {key}: {value.item():.4f}')
    
    print('\n' + '-'*50 + '\n')
    
    # Test MSN EMD through wrapper
    print('2. Testing MSN EMD through wrapper:')
    emd_wrapper = emdModule_wrapper()
    dist_wrapper, assignment_wrapper = emd_wrapper(a, b, eps=0.002, iters=10000)
    print(f'\nMSN EMD Results (via wrapper):')
    print(f'  dist shape: {dist_wrapper.shape}')
    print(f'  assignment shape: {assignment_wrapper.shape}')
    print(f'  Mean distance: {dist_wrapper.mean().item():.6f}')
    print(f'  Std distance: {dist_wrapper.std().item():.6f}')
    
    print('\n' + '-'*50 + '\n')
    
    # Test PyTorchEMD through wrapper
    print('3. Testing PyTorchEMD through wrapper:')
    a_transpose = a.transpose(1, 2)  # Convert to (B, 3, N) format
    b_transpose = b.transpose(1, 2)
    cost_wrapper = earth_mover_distance_wrapper(a_transpose, b_transpose, transpose=True)
    print(f'\nPyTorchEMD Results (via wrapper):')
    print(f'  cost shape: {cost_wrapper.shape}')
    print(f'  Mean cost: {cost_wrapper.mean().item():.6f}')
    print(f'  Std cost: {cost_wrapper.std().item():.6f}')
    print(f'  Min cost: {cost_wrapper.min().item():.6f}')
    print(f'  Max cost: {cost_wrapper.max().item():.6f}')
    
    print('\n' + '-'*50 + '\n')
    
    # Test ChamferDistance through wrapper
    print('4. Testing ChamferDistance through wrapper:')
    chamfer_wrapper = chamfer_3DDist_wrapper()
    dist1_wrapper, dist2_wrapper, idx1_wrapper, idx2_wrapper = chamfer_wrapper(a, b)
    print(f'\nChamferDistance Results (via wrapper):')
    print(f'  dist1 shape: {dist1_wrapper.shape}')
    print(f'  dist2 shape: {dist2_wrapper.shape}')
    print(f'  Mean dist1: {dist1_wrapper.mean().item():.6f}')
    print(f'  Mean dist2: {dist2_wrapper.mean().item():.6f}')
    
    print('\n✓ Wrapper test passed')
else:
    print('⊘ Wrapper test skipped (not available or no CUDA)')

Testing pytorch_pcd_metrics wrapper...

1. Testing StructuralLosses through wrapper:

L-GAN and 1-NN Metrics (via wrapper):
  lgan_mmd-CD: 0.0044
  lgan_cov-CD: 0.5938
  lgan_mmd_smp-CD: 0.0044
  lgan_mmd-EMD: 0.0776
  lgan_cov-EMD: 0.5938
  lgan_mmd_smp-EMD: 0.0777
  1-NN-CD-acc_t: 0.3125
  1-NN-CD-acc_f: 0.5312
  1-NN-CD-acc: 0.4219
  1-NN-EMD-acc_t: 0.3750
  1-NN-EMD-acc_f: 0.3125
  1-NN-EMD-acc: 0.3438

--------------------------------------------------

2. Testing MSN EMD through wrapper:

MSN EMD Results (via wrapper):
  dist shape: torch.Size([32, 2048])
  assignment shape: torch.Size([32, 2048])
  Mean distance: 0.004985
  Std distance: 0.005404

--------------------------------------------------

3. Testing PyTorchEMD through wrapper:

PyTorchEMD Results (via wrapper):
  cost shape: torch.Size([32])
  Mean cost: 43.815937
  Std cost: 8.385543
  Min cost: 30.063068
  Max cost: 68.109688

--------------------------------------------------

4. Testing ChamferDistance through wrap